# Import libraries

In [8]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, BatchNormalization, Dropout
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
import keras
from datetime import datetime

# Load training dataset

In [2]:
train_dataframe = pd.read_csv('dataset/1. istanbul/train_data.csv')

# Visualize some training data

In [3]:
train_dataframe.head()

,Subject ID,Jitter (local),"Jitter (local, absolute)",Jitter (rap),Jitter (ppq5),Jitter (ddp),Shimmer (local),"Shimmer (local, dB)",Shimmer (apq3),Shimmer (apq5),...,Maximum pitch,Number of pulses,Number of periods,Mean period,Standard deviation of period,Fraction of locally unvoiced frames,Number of voice breaks,Degree of voice breaks,UPDRS,Class information
0,1,1.488,0.000090,0.900,0.794,2.699,8.334,0.779,4.517,4.609,...,187.576,160,159,0.006065,0.000416,0.000,0,0.000,23,1
1,1,0.728,0.000038,0.353,0.376,1.059,5.864,0.642,2.058,3.180,...,234.505,170,169,0.005181,0.000403,2.247,0,0.000,23,1
2,1,1.220,0.000074,0.732,0.670,2.196,8.719,0.875,4.347,5.166,...,211.442,1431,1427,0.006071,0.000474,10.656,1,0.178,23,1
3,1,2.502,0.000123,1.156,1.634,3.469,13.513,1.273,5.263,8.771,...,220.230,94,92,0.004910,0.000320,0.000,0,0.000,23,1
4,1,3.509,0.000167,1.715,1.539,5.145,9.112,1.040,3.102,4.927,...,225.162,117,114,0.004757,0.000380,18.182,1,13.318,23,1


# Prepare training data

In [4]:
train_x = train_dataframe.drop(['Subject ID', 'Class information', 'UPDRS'], axis=1)
train_x = train_x.as_matrix()
print(train_x.shape)

(1040, 26)


# Prepare training labels

In [5]:
train_y = train_dataframe['Class information']
train_y = train_y.as_matrix()
print(train_y.shape)

(1040,)


# Prepare test data

In [6]:
test_dataframe = pd.read_csv('dataset/1. istanbul/test_data.csv')
test_x = test_dataframe.drop(['Subject ID', 'Class information'], axis=1)
test_x = test_x.as_matrix()
print(test_x.shape)

(168, 26)


# Prepare test labels

In [7]:
test_y = test_dataframe['Class information']
test_y = test_y.as_matrix()
print(test_y.shape)

(168,)


# Prepare model for training

In [ ]:
model = Sequential()
model.add(BatchNormalization(input_shape = (train_x.shape[1],)))

model.add(Dense(units = 256, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Dense(units = 128, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Dense(units = 64, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Dense(units = 32, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())

model.add(Dense(units = 1))
model.add(Activation("sigmoid"))

model.summary()
SVG(model_to_dot(model, show_layer_names=False, show_shapes=True).create(prog='dot', format='svg'))

# Train the model

In [ ]:
date = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir='Tensorboard/' + date + '/')
model.compile(optimizer=keras.optimizers.RMSprop(), loss=keras.losses.binary_crossentropy, metrics=['accuracy'])
model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=5000, batch_size=20, callbacks=[tensorboard_callback])

# The End